In [1]:
# 예제 7-1 selenium으로 다나와 검색 결과 URL에 접속
from selenium import webdriver
driver = webdriver.Chrome('C:/Users/yj/Desktop/datasalon/chromedriver.exe')
url = 'https://search.shopping.naver.com/search/all?query=%EB%85%B8%ED%8A%B8%EB%B6%81&bt=-1&frm=NVSCDIG'
driver.get(url)

C:\Users\yj\AppData\Local\Temp/ipykernel_2376/2295078348.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/yj/Desktop/datasalon/chromedriver.exe')


In [3]:
from bs4 import BeautifulSoup

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [4]:
notebooks = soup.select('li.basicList_item__2XT81 > div.basicList_inner__eY_mq > div.basicList_info_area__17Xyo')
len(notebooks)

5

In [5]:
name = soup.select('div.basicList_title__3P9Q7> a')[0].text.strip()
print(name)

LG전자 LG그램 17Z95P-GA76K


In [6]:
price = soup.select('strong.basicList_price__2r23_ > span > span.price_num__2WUXn')[0].text.strip()
## 최저~원으로 뜨는거 정확하게 ~ 부분만 추출
price.split('원')[0].replace(',','')

'2329000'

In [36]:
spec = soup.select('div.basicList_detail_box__3ta3h')[0].text.strip()
spec

'화면크기 : 39.62cm(15.6인치)|무게 : 1.81kg|종류 : 코어i5 11세대|운영체제 : 리눅스|CPU : 코어i5-1135G7|칩셋 제조사 : 인텔|코어종류 : 쿼드코어|코드명 : 타이거레이크|CPU속도 : 2.4GHz|터보부스트 : 4.2GHz|램 : 8GB|메모리 타입 : DDR4|인텔 GPU : Iris Xe Graphics|그래픽 메모리 : 시스템 메모리 공유|해상도 : 1920x1080(FHD)|패널 : PLS패널(광시야각)|화면비율 : 16대9|베젤 : 슬림베젤|무선랜 : 802.11 ax(Wi-Fi6)|블루투스 : 블루투스5.0|유선랜 : 유선랜(기가비트)|영상출력 : HDMI|단자 : USB Type C, USB3.0, USB2.0|카드 슬롯 : MicroSD카드|부가기능 : 웹캠|SSD : 256GB|SSD 인터페이스 : NVMe|배터리용량 : 43Wh|충전단자 : DC|웹카메라 : 전면|스피커 : 스테레오스피커|품목 : 노트북|두께 : 1.88cm|조명 : LED백라이트|입력장치 : 숫자키패드|보안기능 : TPM'

In [ ]:
notebook_list = []

for i in notebooks:
    try:
        ## 상품명 가져오기
        name = i.select('div.basicList_title__3P9Q7> a')[0].text.strip()
        ## 가격
        price = i.select('strong.basicList_price__2r23_ > span > span.price_num__2WUXn')[0].text.strip().split('원')[0].replace(',','')
        ## 스펙
        spec = i.select('div.basicList_detail_box__3ta3h')[0].text.strip()
        ## append
        notebook_list.append([name,price,spec])
    except:
        pass

print(len(notebook_list))
notebook_list

In [38]:
## 위 코드 함수

def get_notebooks(ntb):
    notebook_list = []

    for i in notebooks:
        try:
            ## 상품명 가져오기
            name = i.select('div.basicList_title__3P9Q7> a')[0].text.strip()
            ## 가격
            price = i.select('strong.basicList_price__2r23_ > span > span.price_num__2WUXn')[0].text.strip().split('원')[0].replace(',','')
            ## 스펙
            spec = i.select('div.basicList_detail_box__3ta3h')[0].text.strip()
            ## append
            notebook_list.append([name,price,spec])
        except:
            pass
    return notebook_list

In [41]:
# 함수 정상 작동
ntb = soup.select('li.basicList_item__2XT81 > div.basicList_inner__eY_mq > div.basicList_info_area__17Xyo')
len(get_notebooks(ntb))

46

검색 결과 페이지 url 분석

1페이지
https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex=1&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list

2페이지
https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex=2&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list

3페이지
https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=%EB%85%B8%ED%8A%B8%EB%B6%81&pagingIndex=2&pagingSize=40&productSet=total&query=%EB%85%B8%ED%8A%B8%EB%B6%81&sort=rel&timestamp=&viewType=list

다른 검색어 검색(강아지사료)
https://search.shopping.naver.com/search/all?frm=NVSHATC&origQuery=%EA%B0%95%EC%95%84%EC%A7%80%EC%82%AC%EB%A3%8C&pagingIndex=1&pagingSize=40&productSet=total&query=%EA%B0%95%EC%95%84%EC%A7%80%EC%82%AC%EB%A3%8C&sort=rel&timestamp=&viewType=list

주소 비교해보면 origiQuery, pagingIndex, query 부분이 달라지는 것을 볼 수 있음.

In [47]:
def get_search_page_url(keyword,page):
    return 'https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery={}&pagingIndex={}&pagingSize=40&productSet=total&query={}&sort=rel&timestamp=&viewType=list'.format(keyword,page,keyword)

# keyword = '강아지사료'
# page=1
url = get_search_page_url('강아지사료',1)
print(url)


https://search.shopping.naver.com/search/all?frm=NVSCDIG&origQuery=강아지사료&pagingIndex=1&pagingSize=40&productSet=total&query=강아지사료&sort=rel&timestamp=&viewType=list


In [48]:
import time
from tqdm import tqdm_notebook
total_page = 3
for page in tqdm_notebook(range(1,total_page+1)):
    time.sleep(5)


C:\Users\yj\AppData\Local\Temp/ipykernel_11492/1237623251.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1,total_page+1)):


  0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
### 여러 페이지 데이터 수집

keyword = '노트북'
total_page = 10
notebooks_totaldata = []

# 진행 정도를 표현하는 tqdm을 적용
for page in tqdm_notebook(range(1,total_page+1)):
    # 1. 검색 페이지 이동
    url = get_search_page_url('노트북',page)
    driver.get(url)

    # 2. 페이지가 로딩 완료되기 위한 시간으로 5초를 할당
    time.sleep(5)

    # 3. 현재 페이지의 html 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 4. 상품 정보 추출
    notebooks = soup.select('li.basicList_item__2XT81 > div.basicList_inner__eY_mq > div.basicList_info_area__17Xyo')
    notebook_list = get_notebooks(notebooks)

    ## 5. 추출 데이터 저장
    notebooks_totaldata = notebooks_totaldata + notebook_list

C:\Users\yj\AppData\Local\Temp/ipykernel_11492/545302557.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(1,total_page+1)):


  0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
## 엑셀파일로 저장
import pandas as pd
data = pd.DataFrame(notebooks_totaldata)
data.to_excel('C:/Users/yj/Desktop/notebooks.xlsx', index = False)
data.columns = (['상품명','가격','스펙'])
data.head()

NameError: name 'notebooks_totaldata' is not defined

In [ ]:
company_list = []
notebook_list = []
for i in data['상품명']:
    company_list.append(i.split(' ')[0])
    notebook_list.append(i.split(' ')[1])

company_list

In [62]:
spec

'화면크기 : 35.81cm(14.1인치)|무게 : 1.3kg|종류 : 셀러론|운영체제 : 윈도우10 홈|CPU : 셀러론-N4020|칩셋 제조사 : 인텔|코어종류 : 듀얼코어|코드명 : 제미니레이크|CPU속도 : 1.1GHz|터보부스트 : 2.8GHz|램 : 4GB|메모리 타입 : DDR4L|인텔 GPU : UHD600|그래픽 메모리 : 시스템 메모리 공유|해상도 : 1920x1080(FHD)|패널 : IPS패널(광시야각)|화면비율 : 16대9|베젤 : 슬림베젤|특징 : 가벼운노트북|bit유형 : 64|무선랜 : 802.11 ac(Wi-Fi5)|블루투스 : 블루투스5.0|영상출력 : microHDMI|단자 : USB3.0, USB2.0|부가기능 : 웹캠, 팬리스(무소음)|SSD : 128GB|SSD 인터페이스 : M.2|ODD : 광Drive없음|배터리 타입 : 리튬폴리머|사용시간 : 최대11시간|저전력 : O|무상AS : 1년|품목 : 노트북|두께 : 1.5cm|색상 : 화이트'

In [63]:
# 스펙 분리
spec_list = data['스펙'][0].split('|')
print(spec_list)

['화면크기 : 39.62cm(15.6인치)', '무게 : 1.81kg', '종류 : 코어i5 11세대', '운영체제 : 리눅스', 'CPU : 코어i5-1135G7', '칩셋 제조사 : 인텔', '코어종류 : 쿼드코어', '코드명 : 타이거레이크', 'CPU속도 : 2.4GHz', '터보부스트 : 4.2GHz', '램 : 8GB', '메모리 타입 : DDR4', '인텔 GPU : Iris Xe Graphics', '그래픽 메모리 : 시스템 메모리 공유', '해상도 : 1920x1080(FHD)', '패널 : PLS패널(광시야각)', '화면비율 : 16대9', '베젤 : 슬림베젤', '무선랜 : 802.11 ax(Wi-Fi6)', '블루투스 : 블루투스5.0', '유선랜 : 유선랜(기가비트)', '영상출력 : HDMI', '단자 : USB Type C, USB3.0, USB2.0', '카드 슬롯 : MicroSD카드', '부가기능 : 웹캠', 'SSD : 256GB', 'SSD 인터페이스 : NVMe', '배터리용량 : 43Wh', '충전단자 : DC', '웹카메라 : 전면', '스피커 : 스테레오스피커', '품목 : 노트북', '두께 : 1.88cm', '조명 : LED백라이트', '입력장치 : 숫자키패드', '보안기능 : TPM']


In [75]:
## 하면크기, 무게, cpu 추출

for i in spec_list:
    if '화면크기' in i:
        screen_size = i
    elif '무게' in i:
        weight = i
    elif 'CPU ' in i:
        cpu = i